In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
home_dir = '/Users/zehaiwang/Downloads/2018NBA_Hackathon_Files/Basketball Analytics/'

In [3]:
file_list = glob.glob(home_dir+'*.txt')
print ([x.split('/')[-1] for x in file_list])

['NBA Hackathon - Play by Play Data Sample (50 Games).txt', 'NBA Hackathon - Event Codes.txt', 'NBA Hackathon - Game Lineup Data Sample (50 Games).txt']


In [4]:
pbyp_df = pd.read_csv(file_list[0],sep='\t')
eventcode_df = pd.read_csv(file_list[1], sep = '\t')
game_df = pd.read_csv(file_list[2],sep='\t')

## Event Code gives mapping from Event_Msg_type and Action_Type to their decription

_ free throw and No shot_ share the same code

In [5]:
event_map = eventcode_df[['Event_Msg_Type','Event_Msg_Type_Description']].drop_duplicates().sort_values(by = 'Event_Msg_Type')
print (event_map)

     Event_Msg_Type                Event_Msg_Type_Description
0                 1  Made Shot                               
32                2  Missed Shot                             
65                3  Free Throw                              
146               3  No Shot                                 
72                4  Rebound                                 
74                5  Turnover                                
100               6  Foul                                    
116               7  Violation                               
121               8  Substitution                            
126               9  Timeout                                 
131              10  Jump Ball                               
132              11  Ejection                                
135              12  Start Period                            
136              13  End Period                              


### important event for analysis purposes

8 substitution
3 free throw
1 made shot

In [6]:
eventcode_df[eventcode_df.Event_Msg_Type==3].sort_values('Action_Type')

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
146,3,0,No Shot,NaN
65,3,10,Free Throw,Free Throw 1 of 1
66,3,11,Free Throw,Free Throw 1 of 2
67,3,12,Free Throw,Free Throw 2 of 2
68,3,13,Free Throw,Free Throw 1 of 3
142,3,14,Free Throw,Free Throw 2 of 3
69,3,15,Free Throw,Free Throw 3 of 3
70,3,16,Free Throw,Free Throw Technical
71,3,17,Free Throw,Free Throw Clear Path
159,3,18,Free Throw,Free Throw Flagrant 1 of 2


In [7]:
free_throw = eventcode_df[eventcode_df.Event_Msg_Type==3].sort_values('Action_Type')
# first throw of 
free_throw[free_throw.Action_Type_Description.str.contains('1 of', na=False)]


,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
65,3,10,Free Throw,Free Throw 1 of 1
66,3,11,Free Throw,Free Throw 1 of 2
68,3,13,Free Throw,Free Throw 1 of 3
159,3,18,Free Throw,Free Throw Flagrant 1 of 2
157,3,20,Free Throw,Free Throw Flagrant 1 of 1
147,3,21,Free Throw,Free Throw Technical 1 of 2
154,3,25,Free Throw,Free Throw Clear Path 1 of 2
270,3,27,Free Throw,Free Throw Flagrant 1 of 3


### Calculate the 5 player on court during each event
Start from Event_Msg_ID 12 Start Period
Before the first substitution 8


In [8]:
game_df.head()

,Game_id,Period,Person_id,Team_id,status
0,021fd159b55773fba8157e2090fe0fe2,1,881f83d2dee3f18c7d1751659406144e,012059d397c0b7e5a30a5bb89c0b075e,A
1,021fd159b55773fba8157e2090fe0fe2,1,27ea17a8685c4919f157e83fe9cb2d9e,cff694c8186a4bd377de400e4f60fe47,A
2,021fd159b55773fba8157e2090fe0fe2,1,57bbd7e30bc694aeee9ee40c583e6811,cff694c8186a4bd377de400e4f60fe47,A
3,021fd159b55773fba8157e2090fe0fe2,1,cec898a1d355dbfbad8c760615fde1af,012059d397c0b7e5a30a5bb89c0b075e,A
4,021fd159b55773fba8157e2090fe0fe2,1,33963fe856a1523ff46438ba07d1d99f,cff694c8186a4bd377de400e4f60fe47,A


#### We can have period larger than 4

In [9]:
set(game_df.Period.tolist())

{1, 2, 3, 4, 5}

In [10]:
# status not useful
set(game_df.status.tolist())

{'A'}

In [11]:
by_team = game_df.groupby(['Game_id',"Period",'Team_id'])
person_per_team_game = set(by_team['Person_id'].count().tolist())
print (person_per_team_game)
if len(person_per_team_game)==1:
    print ('All the team have ',person_per_team_game, 'Players at the initiate per period')

{5}
All the team have  {5} Players at the initiate per period


In [12]:
def to_set(col):
    return list(set(col))

## create a df of first 5 player at the beginning of each period

In [13]:
l = by_team.Person_id.apply(to_set)
shoufa = l.reset_index()

## important columns in PbyP dataframe:

__WC_Time__: Real_world time, in unit of 0.1s

__PC_Time__: Countdown watch time per quarter, 0-7200 in 0.1s

__OPtion1__: Scores for each event, (1-3) pts

__Person1, Person2__: for assisst event: Person1 is the shot maker;
                      for substitution: Person1 is the ID leaving the game

### Check the persion in game 1 period 1

021fd159b55773fba8157e2090fe0fe2	1	012059d397c0b7e5a30a5bb89c0b075e

__All the player of team 1 on court during the game__

who is on which team?

In [14]:
pbyp_df.head()

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Team_id_type
0,021fd159b55773fba8157e2090fe0fe2,0,12,1,546427,7200,0,0,0,0,1473d70e5646a26de3c52aa1abd85b1f,6bcf6c1f8c373d25fca1579bc4464a91,6bcf6c1f8c373d25fca1579bc4464a91,0
1,021fd159b55773fba8157e2090fe0fe2,1,10,1,546495,7200,0,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,89706b99ddd00dc05d37ef5cafc04276,307beab25b1021a548b4a47550bc4b25,2
2,021fd159b55773fba8157e2090fe0fe2,2,2,1,546665,7050,1,3,0,0,012059d397c0b7e5a30a5bb89c0b075e,cec898a1d355dbfbad8c760615fde1af,6bcf6c1f8c373d25fca1579bc4464a91,2
3,021fd159b55773fba8157e2090fe0fe2,3,4,1,546714,6960,0,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,307beab25b1021a548b4a47550bc4b25,6bcf6c1f8c373d25fca1579bc4464a91,2
4,021fd159b55773fba8157e2090fe0fe2,6,6,1,546886,6920,4,0,0,0,cff694c8186a4bd377de400e4f60fe47,c00264c3114d23bac482e9de50fb7d28,89706b99ddd00dc05d37ef5cafc04276,3


#### 6bcf6c1f8c373d25fca1579bc4464a91 is the default person_id for no-second-person in the event
#### Team_id type: 2 for home, 3 for away team, 0 is default team (judges)

In [15]:
# Game list
Game_list = list(set(pbyp_df.Game_id.tolist()))
print ("Here is totally: ",len(Game_list), "games")

Here is totally:  50 games


In [16]:
def team_set(col): # rule out the default team
    return list(set(i for i in col if i!="1473d70e5646a26de3c52aa1abd85b1f"))

# Teams associated with each game
tmp = pbyp_df.groupby("Game_id").Team_id.apply(team_set)
tmp.head()

game_teams = tmp.to_dict()

In [17]:
# dictionary of player and score
# demo
dic_players = {}
Game_i = '021fd159b55773fba8157e2090fe0fe2'
team1_shoufa = shoufa[(shoufa.Game_id== Game_i)&(shoufa.Period == 1)]
print ("first 5 player on team 1:", team1_shoufa.Person_id[0])
print ("first 5 player on team 2:", team1_shoufa.Person_id[1])

first 5 player on team 1: ['881f83d2dee3f18c7d1751659406144e', '89706b99ddd00dc05d37ef5cafc04276', 'cec898a1d355dbfbad8c760615fde1af', 'a99f44bbff39e352191a870e17f04537', '2b313e2bcef0268bc8e9415132ba9997']
first 5 player on team 2: ['27ea17a8685c4919f157e83fe9cb2d9e', '57bbd7e30bc694aeee9ee40c583e6811', '33963fe856a1523ff46438ba07d1d99f', '307beab25b1021a548b4a47550bc4b25', 'c00264c3114d23bac482e9de50fb7d28']


In [18]:
# Special free throw cases
pbyp_df[(pbyp_df.Event_Msg_Type==3)&(pbyp_df.Action_Type.isin(range(16,30)))]

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Team_id_type
178,021fd159b55773fba8157e2090fe0fe2,210,3,2,570814,3670,16,1,0,0,cff694c8186a4bd377de400e4f60fe47,33963fe856a1523ff46438ba07d1d99f,6bcf6c1f8c373d25fca1579bc4464a91,3
331,021fd159b55773fba8157e2090fe0fe2,391,3,3,602515,1460,16,1,0,0,012059d397c0b7e5a30a5bb89c0b075e,89706b99ddd00dc05d37ef5cafc04276,6bcf6c1f8c373d25fca1579bc4464a91,2
1345,06bb1d31c63891e2580ff12e4e6505b4,441,3,4,637313,1420,16,1,0,0,cc5e0f4efd0f5410547d4e73f38f7811,d8e4f0fd2e836518e681c99a3f9e4411,6bcf6c1f8c373d25fca1579bc4464a91,3
1463,07e76f7482773e81e2351d1692e9e5bb,97,3,1,561020,1930,16,1,0,0,8e90edc4eb908903bcbb567f92488049,9075d598a348b290d9c426e41662e593,6bcf6c1f8c373d25fca1579bc4464a91,3
1638,07e76f7482773e81e2351d1692e9e5bb,323,3,3,598551,5840,16,2,0,0,78cb4fd2222d4d6c7ce0a53cf99b4b14,f2e41e7ce25aff54b85ee5ab66f69187,6bcf6c1f8c373d25fca1579bc4464a91,2
2131,0868dee930f69a54541d4ae88b841a37,276,3,3,607797,6640,16,2,0,0,78cb4fd2222d4d6c7ce0a53cf99b4b14,5f2d1b7a31eb9117654c9d056093bfa5,6bcf6c1f8c373d25fca1579bc4464a91,3
2152,0868dee930f69a54541d4ae88b841a37,305,3,3,612541,5330,16,1,0,0,3cd0b15957ceb80f5125bef8bd1bbea7,45e7a758a91a254ff04756562e740f2d,6bcf6c1f8c373d25fca1579bc4464a91,2
2179,0868dee930f69a54541d4ae88b841a37,340,3,3,616202,3750,16,2,0,0,78cb4fd2222d4d6c7ce0a53cf99b4b14,bc77b0f38cb51b2cfe8552000e585cbf,6bcf6c1f8c373d25fca1579bc4464a91,3
3779,1eab6189ad9ab246c197575a8c4eebe5,26,3,1,675111,5690,16,2,0,0,3a30c29e8a04b4f1335ee8874c526a6c,c5a353e301078ea1cb4e8bc8fede12e9,6bcf6c1f8c373d25fca1579bc4464a91,3
4076,1eab6189ad9ab246c197575a8c4eebe5,367,3,3,728103,1840,16,2,0,0,cd45058739ed0ac8229849c6249aad48,c5a353e301078ea1cb4e8bc8fede12e9,6bcf6c1f8c373d25fca1579bc4464a91,2


## For special Free throw, we do not have to worry about "substitution" in between.

In [19]:
direct_free_throw = list(range(16,30))
direct_free_throw.append(10)
print (direct_free_throw)

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 10]


In [20]:
count_free_throw = [11, 13]

### For regular free throw, we can just consider the first throw
#### once the first throw start we consider total score of the throw
65	3	10	Free Throw	Free Throw 1 of 1

66	3	11	Free Throw	Free Throw 1 of 2

67	3	12	Free Throw	Free Throw 2 of 2

68	3	13	Free Throw	Free Throw 1 of 3

142	3	14	Free Throw	Free Throw 2 of 3

69	3	15	Free Throw	Free Throw 3 of 3

In [21]:
# function to calculate score in the free throw round
def free_throw_score(row, num_of_throw): # input the row_index of the first free throw
    current_throw = 1
    cur_row = int(row.name)
    score = row.Option1
    while (current_throw < num_of_throw):
        cur_row = cur_row + 1
        if pbyp_df.iloc[cur_row,2]==3:
            current_throw += 1
            score = score + pbyp_df.iloc[cur_row,7]
    return score

In [22]:
def plus_minus(score_team, loss_team, pts, dic):
    # input the active persons in the score_team and loss_team:
    for person in score_team:
        if person in dic.keys():
            dic[person] = dic[person]+pts
        else:
            dic[person] = pts
    for person in loss_team:
        if person in dic.keys():
            dic[person] = dic[person]-pts
        else:
            dic[person] = -pts

In [23]:
def event_judge(row, team_active_1, team_active_2, dic):
    # scores
#         1  Made Shot                                                           
#         3  Free Throw                                                                                                                     
    if row.Event_Msg_Type == 1:
        pts = row.Option1
        if row.Person1 in team_active_1:
            plus_minus(team_active_1, team_active_2, pts, dic) # def plus_minus(score_team, loss_team, pts, dic):
        elif row.Person1 in team_active_2:
            plus_minus(team_active_2, team_active_1, pts, dic)
        else:
            print ("Error: This person %s should not be on court"%row.Person1)
    elif (row.Event_Msg_Type == 3):
        # direct free throw
        if (row.Action_Type in direct_free_throw):
            pts = row.Option1
        elif row.Action_Type ==11:
            pts = free_throw_score(row, 2)
        elif row.Action_Type ==13:
            pts = free_throw_score(row, 3)
        else: # ignore 2 of 2, 2 of 3, 3 of 3 cases:
            pts = 0
        if row.Person1 in team_active_1:
            plus_minus(team_active_1, team_active_2, pts, dic) # def plus_minus(score_team, loss_team, pts, dic):
        elif row.Person1 in team_active_2:
            plus_minus(team_active_2, team_active_1, pts, dic)
        else:
            print ("Error: This person %s should not be on court"%row.Person1)
#         8  Substitution    
    elif row.Event_Msg_Type == 8:
        player_out = row.Person1
        player_in = row.Person2
        if (player_out in team_active_1) and (player_in not in team_active_1):
            team_active_1.remove(player_out)
            team_active_1.append(player_in)
            assert(len(team_active_1)==5)
        elif (player_out in team_active_2) and (player_in not in team_active_2):
            team_active_2.remove(player_out)
            team_active_2.append(player_in)
            assert(len(team_active_2)==5)
        else:
            print ("Error: Substitution problem in row %s!"%row.index)

In [24]:
result = []
for game_i in Game_list:
    # loop_over games
    # initiate two teams in the game
    team_1 = game_teams[game_i][0]
    team_2 = game_teams[game_i][1]
     # print ( "two teams in Game %s are:"%index, team_1, team_2)
    dic = {} #initiate socre dict for the game
    # loop over Period of the game
    for period_i in range(1,6):
        df = pbyp_df[(pbyp_df.Game_id==game_i)&(pbyp_df.Period==period_i)] 
        if not df.empty:
            # get initial player of each team
            team_shoufa = shoufa[(shoufa.Game_id== game_i)&(shoufa.Period == period_i)]
            team_active_1 = team_shoufa.Person_id.tolist()[0]
            team_active_2 = team_shoufa.Person_id.tolist()[1]
            # loop over the df of current game and period
            df.apply(event_judge, args =(team_active_1, team_active_2, dic), axis=1)
    tmp_result = pd.Series(dic, name='Player_Plus/Minus')
    tmp_result.index.name = 'Player_ID'
    tmp_result = tmp_result.reset_index()
    tmp_result['Game_ID'] = game_i
    result.append(tmp_result)

### Check the sum of all players in a game should be 0

In [25]:
# asseble all the dfs generated
result_df = pd.concat(result, axis=0)

In [26]:
# check the sum of all score should be 0
result_df.groupby('Game_ID')["Player_Plus/Minus"].sum()

Game_ID
021fd159b55773fba8157e2090fe0fe2    0
03a31e84b194d6c8a2eab5d70ba67acf    0
06bb1d31c63891e2580ff12e4e6505b4    0
07e76f7482773e81e2351d1692e9e5bb    0
0868dee930f69a54541d4ae88b841a37    0
09d46e3d7a8253b7209100650b5afaeb    0
13ced855d491384876c6ab807bd1d3db    0
15d76177caa6022156e83774c2e054d3    0
1eab6189ad9ab246c197575a8c4eebe5    0
1f9e3cb05c031986cf8bc7c0a84cc517    0
2bf4ac0ed9ac1aee8767134d62b34dfe    0
2faabfa663f4dfb9ed83a1482088e092    0
3152e9c330ce200cc189ae64ebdf41fc    0
33f631fec90cc1f08bb16cff5ed52f9b    0
345f285f7c654bf7a03e940a9bc923c3    0
3b753670a0d0df2c35c7ce2e0bc94e6c    0
56ccfaf0adead6f4c7236a01ca0cfbdc    0
598a55c8bf052d039d0fb4ff1a62f98a    0
7fad2269ee0d11ae5069ff23ecb25913    0
87d1574f478f37d13789284b96b4f6cb    0
88012a99d7fd1c169e2360aa5cdf8bfa    0
896bcac9b0f35b250c9fb7a6325d8599    0
8fd2943309c7accc78b41e047e1e9393    0
9db75b921076b5848cba7ca36436147c    0
9dce47bc3cbabd451a9bab758ca5467f    0
a109cef8327feae75c3e4885cb4c6e2d    0
a322

In [27]:
result_df[['Game_ID','Player_ID','Player_Plus/Minus']].reset_index(drop=True).to_csv('Nubility_Q1_BBALL.csv')

### Final result saved to result file